# SNSD Song Analysis

## Objective
Analyse song data of korean girl group SNSD from 2007 to 2014 (OT9 period + OT8 song versions of the same period), and find patterns in screentime and lyric distribution in songs and their respective music videos (if any).

## How to get it organized

### Screentime dataframe
The screentime dataframe will be manually gathered from video files data. Each video will have all of its frame extracted and counted for member presences, and then converted to time using the FPS of each video (in process). Final data will contain summaries for solo, centre, side and insignificant times in story and dance categories for each member.

### Lyrics dataframe
The lyrics dataframe will be obtained using a full lyrics index with color coding for each member. Files will be obtained programatically from colorcodedlyrics.com and then cleaned to get each line with its corresponding singing member. Final data will contain summaries for total time, number of individual parts, number of words for each member.

## Building the lyrics dataframe

In [1]:
import requests
import os
from bs4 import BeautifulSoup

### Get links for all song lyrics

In [3]:
# making the soup with the lyrics index

url = 'https://colorcodedlyrics.com/2012/02/snsd_lyrics_index'
response = requests.get(url)

name = url.split('/')[-1] + '.txt'
with open(name, mode = 'wb') as file:
    file.write(response.content)

In [4]:
# getting the links for all lyric pages

with open(name, encoding='utf8') as file:
    soup = BeautifulSoup(file, features='html5lib')
    
    lyrics_index = soup.find('table', attrs='indexlyrics')
    
    links = []
    for link in lyrics_index.find_all('a', href=True):
        links.append(link['href'])

### Cleaning the list

In [4]:
# remove all links after The Boys, but keep HaHaHa, Visual Dreams and Chocolate Love

to_keep = ['https://colorcodedlyrics.com/2013/02/girls-generation-sonyeosidae-hahaha-samsung-cf', 
           'https://colorcodedlyrics.com/2011/02/08/snsd-visual-dream-cc-lyrics', 
           'https://colorcodedlyrics.com/2010/04/28/snsd-chocolate-love-color-coded-lyrics/']

links.index('https://colorcodedlyrics.com/2013/04/girls-generation-feat-snoop-dogg-the-boys') #cutoff link

links = links[0:270]

### Remove all originally non-OT9 songs and duplicates

In [5]:
# list of songs: 7989, talk to me, lost in love, one year later, sailing, cmiyc (kr version), girls (both version)
# remove invalid links (str includes 'preview')

links = [ x for x in links if "7989" not in x and "talk-to-me" not in x and "lost-in-love" not in x and "one-year-later" not in x and "sailing" not in x and "preview" not in x]

In [6]:
links.remove("https://colorcodedlyrics.com/2015/04/girls-generation-sonyeosidae-girls")
links.remove("https://colorcodedlyrics.com/2015/08/girls-generation-girls-japanese-ver")
links.remove("https://colorcodedlyrics.com/2015/04/girls-generation-sonyeosidae-catch-me-if-you-can")

In [7]:
# list of albums: party, lion heart, holiday night

print(links.index('https://colorcodedlyrics.com/2015/08/girls-generation-sonyeosidae-lion-heart'))
print(links.index('https://colorcodedlyrics.com/2017/08/girls-generation-sonyeosidae-light-sky'))

51
72


In [8]:
del links[51:73]

In [9]:
print(links.index('https://colorcodedlyrics.com/2015/07/girls-generation-sonyeosidae-party'))

108


In [10]:
del links[108:110]

In [11]:
print(len(links))

234


In [12]:
#remove duplicates

clean_links = []

for link in links:
    if link not in clean_links:
        clean_links.append(link)

In [13]:
print(len(clean_links))

165


### Download source code for all links

In [ ]:
for link in clean_links:
    response = requests.get(link)
    
    if link[-1] == '/':
        name = link.split('/')[-2] + '.txt'
    else:
        name = link.split('/')[-1] + '.txt'
    
    with open(os.path.join('lyrics', name), mode = 'wb') as file:
        file.write(response.content)

### Scraping lyrics and singer from song files

In [78]:
for file in os.listdir('lyrics'):
    with open(os.path.join('lyrics', file), encoding = 'utf8') as file:
        print(file)
        soup = BeautifulSoup(file, features='html5lib')
        
        ## song name
        name = soup.find('h2', style='text-align: center').contents
        
        ## release date 
        date = soup.find('p', style="text-align: center").contents[-1]
        
        ## member colors      
        colors = ['color: #ff6600', 'color: #3366ff', 'color: #800080', 'color: #ff00ff', 'color: #008000', 'color: #ff0000', 
                  'color: #00ccff', 'color: #ffff00', 'color: #008080']
        
        members = ['Taeyeon', 'Jessica', 'Sunny', 'Tiffany', 'Hyoyeon', 'Yuri', 'Sooyoung', 'Yoona', 'Seohyun']
        
        def check_member(x):
            for color in colors:
                if color in x:
                    position = colors.index(color)
                    singer = members[position]
                    return singer
        
        ## lyrics
        lyrics = soup.find('table', border='0')
        lyrics = str(lyrics)
        lyrics = lyrics.split('</td>')[0]
        lyrics = lyrics.split('\n')
        
        ## correct line breaks
        for i in lyrics:
            if '</span>' in i and '[' not in i:
                position = lyrics.index(i)
                lyrics.remove(i)
                i = i.split('</span>')
                lyrics.insert(position, i)
            elif '<td>' in i:
                position = lyrics.index(i)
                lyrics.remove(i)
                i = i.split('<td>')[-1]
                lyrics.insert(position, i)
                
        color_lyrics = []
        
        ## flatten list
        for i in lyrics:
            if type(i) is list:
                for item in i:
                    color_lyrics.append(item)
            else:
                color_lyrics.append(i)
                
        ## remove extra lines containing '<br/>'
        color_lyrics = [x for x in color_lyrics if x != '<br/>']
        
        ## replace extra tags
        for i in color_lyrics:
            if '<p>' in i or '</p>' in i:
                i.replace('<p>', '')
                i.replace('</p', '')
        
        ## clean start of lines
        space = color_lyrics[-22][0]
        
        for i in color_lyrics:
            if i.startswith(space):
                position = color_lyrics.index(i)
                color_lyrics.remove(i)
                i = i[1:]
                color_lyrics.insert(position, i)
        
        ## printing test for lyrics
#         for i in color_lyrics:
#             print(i + '\n')
        
        ## singing order
        singing_order = []
        
        for line in color_lyrics:
            member_line = check_member(line)
            singing_order.append(member_line)
            
            
        ## printing test for order
        print(singing_order)
            
        # split adlibs for main lines 
    

<_io.TextIOWrapper name='lyrics\\girls-generation-all-my-love-is-for-you.txt' mode='r' encoding='utf8'>
[None, None, None, 'Jessica', 'Jessica', 'Jessica', 'Seohyun', 'Seohyun', 'Seohyun', 'Taeyeon', 'Hyoyeon', 'Tiffany', None, None, None, 'Jessica', 'Jessica', None, None, None, None, None, 'Yuri', 'Tiffany', 'Tiffany', 'Tiffany', 'Sunny', 'Sunny', 'Sunny', 'Seohyun', 'Sooyoung', 'Taeyeon', None, None, None, 'Jessica', 'Jessica', 'Jessica', None, None, 'Tiffany', None, 'Tiffany', None, 'Tiffany', 'Yoona', 'Jessica', 'Jessica', 'Tiffany', None, 'Tiffany', 'Taeyeon', 'Jessica', 'Jessica', None, 'Taeyeon', None, 'Taeyeon', None, 'Taeyeon', None, 'Yuri', None, 'Taeyeon', None, None, 'Sunny', 'Taeyeon', None, 'Taeyeon', None, 'Taeyeon', None, 'Sooyoung']
<_io.TextIOWrapper name='lyrics\\girls-generation-animal.txt' mode='r' encoding='utf8'>
['Seohyun', None, 'Yoona', None, 'Taeyeon', 'Jessica', 'Sunny', 'Tiffany', None, None, None, None, 'Yuri', None, 'Sooyoung', None, 'Jessica', 'Hyoyeon',

<_io.TextIOWrapper name='lyrics\\girls-generation-divine.txt' mode='r' encoding='utf8'>
['Taeyeon', 'Sunny', 'Jessica', None, None, 'Tiffany', None, None, 'Yoona', 'Hyoyeon', 'Taeyeon', 'Taeyeon', 'Sunny', 'Sunny', 'Tiffany', 'Jessica', 'Yoona', 'Seohyun', None, None, 'Tiffany', 'Taeyeon', None, None, 'Sooyoung', 'Yuri', 'Sunny', 'Hyoyeon', 'Jessica', 'Jessica', 'Taeyeon', 'Sunny', 'Sooyoung', 'Seohyun', 'Taeyeon', 'Tiffany', 'Jessica', 'Tiffany', 'Jessica', None, None, None, 'Tiffany', 'Taeyeon', None, 'Tiffany', 'Jessica', None]
<_io.TextIOWrapper name='lyrics\\girls-generation-do-the-catwalk.txt' mode='r' encoding='utf8'>
['Jessica', None, None, None, 'Taeyeon', None, 'Seohyun', None, 'Sooyoung', 'Tiffany', 'Yuri', 'Seohyun', 'Yuri', 'Seohyun', 'Yuri', 'Seohyun', 'Yuri', None, 'Sunny', None, None, 'Sooyoung', None, 'Tiffany', 'Tiffany', 'Taeyeon', 'Hyoyeon', None, 'Jessica', None, 'Sunny', None, 'Taeyeon', None, 'Tiffany', 'Yuri', 'Tiffany', None, 'Yoona', None, None, 'Hyoyeon', Non

IndexError: list index out of range